# Purpose
The purpose of this notebook is estimate the "Mixed Logit B" model of Brownstone and Train (1998) using pytorch.

# Notebook parameters

In [1]:
# Declare paths to where data is or should be stored
DATA_PATH =\
    "../../data/processed/model_ready_car_data.csv"

OUTPUT_PARAM_PATH =\
    "../../models/estimated_mixlb_parameters.csv"

OUTPUT_GRADIENT_PATH =\
    "../../models/estimated_mixlb_gradient.csv"

OUTPUT_HESSIAN_PATH =\
    "../../models/estimated_mixlb_hessian.csv"

# Note needed column names
ALT_ID_COLUMN = 'alt_id'
OBS_ID_COLUMN = 'obs_id'
CHOICE_COLUMN = 'choice'

# Import needed libraries

In [2]:
# Built-in modules
import sys
import time
from collections import OrderedDict

# Third-party modules
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import pylogit as pl
import pylogit.mixed_logit_calcs as mlc

# Local modules
sys.path.insert(0, '../../')
import src.models.mixlb as mixlb
import src.models.torch_utils as utils
from src.hessian import hessian

# Load needed data

In [3]:
car_df = pd.read_csv(DATA_PATH)

# Create the corresponding MNL model
Create a MNL model that has the same design matrix as Mixed Logit B.

In [4]:
# Create specification and name dictionaries
mnl_spec, mnl_names = OrderedDict(), OrderedDict()

for col, display_name in mixlb.DESIGN_TO_DISPLAY_DICT.items():
    mnl_spec[col] = 'all_same'
    mnl_names[col] = display_name

In [5]:
# Instantiate a MNL with the same design matirx as the MIXL.
mnl_model =\
    pl.create_choice_model(data=car_df,
                           alt_id_col=ALT_ID_COLUMN,
                           obs_id_col=OBS_ID_COLUMN,
                           choice_col=CHOICE_COLUMN,
                           specification=mnl_spec,
                           model_type='MNL',
                           names=mnl_names)

# Estimate the model to note reproduction of
# Brownstone & Train's MNL
mnl_model.fit_mle(np.zeros(len(mnl_names)),
                  constrained_pos=[-2, -1])

# Look at the estimation results
mnl_model.get_statsmodels_summary()

Log-likelihood at zero: -8,338.8486
Initial Log-likelihood: -8,338.8486


/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/pylogit/choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  design_matrix = np.hstack((x[:, None] for x in independent_vars))
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.18 seconds.
Final log-likelihood: -7,391.8300


/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/pylogit/base_multinomial_cm_v2.py:1228: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.cov)),
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                4,654
Model:             Multinomial Logit Model   Df Residuals:                    4,631
Method:                                MLE   Df Model:                           23
Date:                     Fri, 12 Jun 2020   Pseudo R-squ.:                   0.114
Time:                             18:02:10   Pseudo R-bar-squ.:               0.111
AIC:                            14,829.660   Log-Likelihood:             -7,391.830
BIC:                            14,977.906   LL-Null:                    -8,338.849
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Neg Price over log(income)           0.1854      0.027      6.796      0.000       0.132       0.239
Range (units: 100mi)                 0.3501      0.027     13.052      0.000       0.298       0.403
Neg Acceleration (units: 0.1sec)     0.7160      0.111      6.472      0.000       0.499       0.933
Neg Top speed (units: 0.01mph)       0.2612      0.081      3.228      0.001       0.103       0.420
Neg Pollution                        0.4441      0.102      4.367      0.000       0.245       0.643
Size                                 0.9345      0.316      2.953      0.003       0.314       1.555
Big enough                           0.1432      0.077      1.853      0.064      -0.008       0.295
Luggage space                        0.5009      0.191      2.623      0.009       0.127       0.875
Neg Operation cost                   0.7679      0.076     10.131      0.000       0.619       0.916
Station availability                 0.4133      0.096      4.294      0.000       0.225       0.602
Sports utility vehicle               0.8201      0.141      5.830      0.000       0.544       1.096
Sports car                           0.6370      0.148      4.298      0.000       0.347       0.928
Station wagon                       -1.4367      0.062    -23.139      0.000      -1.558      -1.315
Truck                               -1.0168      0.049    -20.753      0.000      -1.113      -0.921
Van                                 -0.7989      0.047    -16.864      0.000      -0.892      -0.706
EV                                  -0.1786        nan        nan        nan         nan         nan
Commute < 5 & EV                     0.1983      0.084      2.374      0.018       0.035       0.362
College & EV                         0.4426      0.109      4.058      0.000       0.229       0.656
CNG                                  0.3450        nan        nan        nan         nan         nan
Methanol                             0.3134      0.103      3.051      0.002       0.112       0.515
College & Methanol                   0.2284      0.089      2.576      0.010       0.055       0.402
Non Electric-Vehicle                      0        nan        nan        nan         nan         nan
Non Compressed Natural Gas                0        nan        nan        nan         nan         nan
====================================================================================================
"""

# Initialize the MIXL model

In [6]:
# Instantiate the model
mixl_model = mixlb.MIXLB()

# Create objects for probability and loss calculations

In [7]:
# Create target variables for the loss function
torch_choices =\
    torch.from_numpy(mnl_model.choices.astype(np.float32)).double()

# Get the design matrix from the original and forecast data
orig_design_matrix_np = mnl_model.design
orig_design_matrix =\
    torch.tensor(orig_design_matrix_np.astype(np.float32))

# Get the rows_to_obs and rows_to_mixers matrices.
rows_to_obs =\
    utils.create_sparse_mapping_torch(car_df[mnl_model.obs_id_col].values)
rows_to_mixers =\
    utils.create_sparse_mapping_torch(car_df[mnl_model.obs_id_col].values)

####
# Get the normal random variates.
####
# Determine the number of draws being used for the mixed logit
num_draws = 250
# Determine the number of observations with randomly distributed
# sensitivities
num_mixers = car_df.obs_id.unique().size

# Get the random draws needed for the draws of each coeffcient
# Each element in the list will be a 2D ndarray of shape
# num_mixers by num_draws
normal_rvs_list_np =\
    mlc.get_normal_draws(num_mixers,
                         num_draws,
                         mixl_model.design_info.num_mixing_vars,
                         seed=601)
normal_rvs_list = [torch.from_numpy(x).double() for x in normal_rvs_list_np]


# Create the objective function
Create the function to be used by `scipy.optimize.minimize`.

In [8]:
def make_scipy_closure(
        design,
        obs_mapping,
        mixers_mapping,
        normal_draws,
        targets,
        model,
        loss_func,
        ):
    def closure(params):
        # params -> loss, grad
        # Load the parameters onto the model
        model.set_params_numpy(params)
        # Ensure the gradients are summed starting from zero
        model.zero_grad()
        # Calculate the probabilities
        probabilities =\
            model(design_2d=design,
                  rows_to_obs=obs_mapping,
                  rows_to_mixers=mixers_mapping,
                  normal_rvs_list=normal_draws)
        # Calculate the loss
        loss = loss_func(probabilities, targets)
        # Compute the gradients
        loss.backward()
        # Get the gradient.
        grad = model.get_grad_numpy()
        # Get a value of the loss to pass around.
        loss_val = loss.item()
        return loss_val, grad
    return closure

scipy_objective =\
    make_scipy_closure(orig_design_matrix,
                       rows_to_obs,
                       rows_to_mixers,
                       normal_rvs_list,
                       torch_choices,
                       mixl_model,
                       utils.log_loss)

# Estimate MIXLB

In [9]:
####
# Initialize parameters
####
# Initialize the model parameters to the final estimates from Brownstone & Train (1998),
# taking care of the typo from the published paper.
mean_array =\
    np.array([-1.5983748481622846, #-5.999,
              -0.877,
              -0.302,
              -1.364,
              -0.711,
               1.541,
              -1.748,
               1.563,
              -0.071,
              -0.741,
               0.897,
               0.698,
              -1.508,
              -1.094,
              -0.819,
              -0.905,
               0.359,
               0.770,
               0.621,
               0.476,
               0.335,
               0,
               0])

std_dev_array =\
    np.array([6.808, 5.380, 2.289, 0.971])

paper_estimates_array =\
    np.concatenate((mean_array, std_dev_array), axis=0)

# Set the parameters on the model
mixl_model.set_params_numpy(paper_estimates_array)

In [10]:
####
# Compute initial log-likelihood
####
with torch.no_grad():
    # Compute the MIXL probabilities
    initial_mixl_probs =\
        mixl_model.forward(design_2d=orig_design_matrix,
                           rows_to_obs=rows_to_obs,
                           rows_to_mixers=rows_to_mixers,
                           normal_rvs_list=normal_rvs_list)

    # Compute the MIXL log-likelihood
    initial_mixl_log_likelihood =\
        -1 * utils.log_loss(initial_mixl_probs, torch_choices)

    # Compare the MIXL to MNL log-likelihoods
    msg = 'Initial MIXL: {:,.2f}\nMNL:          {:,.2f}'
    print(msg.format(initial_mixl_log_likelihood.item(), mnl_model.llf))

Initial MIXL: -7,370.09
MNL:          -7,391.83


In [11]:
# Perform the optimization
start_time = time.time()

optimization_results =\
    minimize(scipy_objective,
             paper_estimates_array,
             jac=True,
             method='bfgs')

end_time = time.time()
duration_sec = end_time - start_time
duration_mins = duration_sec / 60.

print('Estimation Time: {:.1f} minutes'.format(duration_mins))

Estimation Time: 10.8 minutes


In [12]:
print('Initial Log-likelihood: {:,.2f}'.format(initial_mixl_log_likelihood))
print('Final Log-Likelihood:    {:,.2f}'.format(optimization_results['fun']))

Initial Log-likelihood: -7,370.09
Final Log-Likelihood:    7,366.56


In [13]:
# Look at the gradient at the final parameters
optimization_results['jac']

array([ 3.02063929e-10,  1.61201999e-07, -4.19390302e-08,  7.47850686e-08,
       -1.82699654e-07,  3.43704994e-08, -1.54603583e-07, -5.11718366e-08,
       -1.05335401e-08,  2.53901248e-07,  1.07174357e-08,  6.49821302e-09,
        2.32708658e-08,  4.43429601e-08, -1.78816608e-08,  3.17923849e-08,
        4.95778456e-08, -1.71724781e-08,  7.90312933e-09, -2.53907315e-07,
       -2.36296594e-07,  0.00000000e+00,  0.00000000e+00, -2.60114483e-08,
       -3.65452061e-08,  1.77963963e-08,  9.52168868e-08])

In [14]:
# Compare the final parameters to their starting values
estimates_df =\
    pd.DataFrame({'initial': paper_estimates_array,
                  'final': optimization_results['x']})
estimates_df

,initial,final
0,-1.598375,-1.511420
1,-0.877000,-0.741690
2,-0.302000,-0.291876
3,-1.364000,-1.472323
4,-0.711000,-0.655299
5,1.541000,1.735805
6,-1.748000,-1.609506
7,1.563000,1.505530
8,-0.071000,-0.027687
9,-0.741000,-0.539973


# Compute the hessian

In [15]:
# Get rid of old gradient computations
mixl_model.zero_grad()

# Compute final probabilities
final_mixl_probs =\
    mixl_model(design_2d=orig_design_matrix,
               rows_to_obs=rows_to_obs,
               rows_to_mixers=rows_to_mixers,
               normal_rvs_list=normal_rvs_list)

# Compute final loss
final_log_likelihood =\
    utils.log_loss(final_mixl_probs, torch_choices)

# Compute the hessian of the loss
hess_start_time = time.time()
final_mixlb_hessian =\
    hessian(final_log_likelihood,
            mixl_model.parameters())
hess_end_time = time.time()
hess_duration_sec = hess_end_time - hess_start_time
hess_duration_mins = hess_duration_sec / 60.
print('Hessian Computation: {:.1f} minutes'.format(hess_duration_mins))

# Get the numpy array corresponding to the hessian
final_mixlb_hessian_array = final_mixlb_hessian.numpy()

# Extract the hessian that excludes the rows and columns
# for the two constrained parameters
desired_rows =\
    np.concatenate((np.arange(0, 21), np.arange(23, 27)), axis=0)
final_mixlb_hessian_core =\
    final_mixlb_hessian_array[np.ix_(desired_rows, desired_rows)]
final_mixlb_hessian_core.shape

Hessian Computation: 18.8 minutes


(25, 25)

# Save the results

In [16]:
# Save the final parameters
estimates_df.final.to_csv(OUTPUT_PARAM_PATH, index=False)
# Save the final gradient
(pd.Series(optimization_results['jac'])
   .to_csv(OUTPUT_GRADIENT_PATH, index=False))
# Save the final hessian
(pd.DataFrame(final_mixlb_hessian_array)
   .to_csv(OUTPUT_HESSIAN_PATH, index=False, header=False))

# Findings
1. The most unexpected finding was that none of the parameter gradients was near zero when using the parameter values from the published article.

2. When optimizing the model to get to a true local maximum of the log-likelihood function, there does not seem to be a huge difference in final results.
The largest parameter change is the increase in the variance of non-EV utility functions.

3. Computing the hessian of the estimated parameters takes a **very** long time.
